In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
# from rsna_retro.train3d import *
from rsna_retro.train_adjacent import *

Loading imports


In [3]:
torch.cuda.set_device(1)

## Train

In [4]:
name = 'train_adjacent_5slice'

In [5]:
dbunch = get_adj_data(512, 128, Meta.splits_stg1, num_adj=2)
learn = get_learner(dbunch, partial(xresnet18, c_in=5))

In [6]:
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.133400,0.156479,0.956919,0.906232,29:25
1,0.115425,0.130152,0.960511,0.920568,28:45
2,0.109224,0.124222,0.963902,0.923547,26:20
3,0.104774,0.109105,0.966792,0.932230,25:44
4,0.098441,0.124953,0.961939,0.908613,25:28
5,0.095702,0.104622,0.968006,0.934853,23:03
6,0.090879,0.101014,0.969326,0.938099,21:59
7,0.085520,0.100963,0.970064,0.937603,21:58
8,0.080912,0.089418,0.972878,0.945025,22:03
9,0.077141,0.086350,0.973601,0.946922,22:00


FileNotFoundError: [Errno 2] No such file or directory: 'models/runs/train_adjacent_5slice-1.pth'

In [ ]:
learn.dbunch = get_adj_data(512, 256, Meta.splits_stg1, num_adj=2)
do_fit(learn, 12, 4e-3)
learn.save(f'runs/{name}-2')

In [ ]:
learn.dbunch = get_adj_data(256, 384, Meta.splits_stg1, num_adj=2, img_dir=path_jpg)
do_fit(learn, 4, 4e-4)
learn.save(f'runs/{name}-3')

## Submission

In [5]:
learn.load(f'runs/{name}-3')
sub_fn = f'subm/{name}'

In [6]:
# learn.dbunch = get_data_gen(Meta.df_tst, bs=512, sz=None, with_aug=False, tst_dir='tst_jpg256')
learn.dbunch = get_adj_test_data(512, sz=None, tst_dir='tst_jpg256')

In [7]:
preds,targs = learn.get_preds()

In [8]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [11]:
api.competition_submit(f'{sub_fn}.csv', f'{name} adjacent 5c', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.80MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [16]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14244524,
 'totalBytes': 27277209,
 'date': '2020-01-21T07:07:06.64Z',
 'description': 'train_adjacent_5c adjacent 5c',
 'errorDescription': None,
 'fileName': 'train_adjacent_5c.csv',
 'publicScore': '0.34569',
 'privateScore': '0.07648',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14244524/14244524.raw'}